In [1]:
import numpy as np
import itertools
import pandas as pd
from tqdm import tqdm
from time import sleep 

In [2]:
n = 10
cards = 5

In [19]:
A = np.array([
    [[1/2],[2/4],[162/294],[54/105],[],[],[],[],[],[]],
    [[],[],[],[],[],[],[],[],[],[]],
    [[],[],[],[],[],[],[],[],[],[]],
    [[],[],[],[],[],[],[],[],[],[]],
    [[],[],[],[],[],[],[],[],[],[]],
    [[],[],[],[],[],[],[],[],[],[]],
    [[],[],[],[],[],[],[],[],[],[]],
    [[],[],[],[],[],[],[],[],[],[]],
    [[],[],[],[],[],[],[],[],[],[]],
    [[],[],[],[],[],[],[],[],[],[]],
])

In [3]:
A = np.random.sample(n*n).reshape(n, n)
for i in range(n):
    for j in range(n):
        if i < j:
            A[i, j] = 1 - A[j, i]
        if i == j:
            A[i, i] = 0.5
pd.DataFrame(A)

,0,1,2,3,4,5,6,7,8,9
0,0.500000,0.524621,0.251550,0.288951,0.022516,0.842196,0.216038,0.157643,0.380458,0.068813
1,0.475379,0.500000,0.157053,0.670516,0.793667,0.172798,0.837687,0.911688,0.530822,0.068988
2,0.748450,0.842947,0.500000,0.945454,0.178560,0.024000,0.456866,0.884550,0.069449,0.016111
3,0.711049,0.329484,0.054546,0.500000,0.695739,0.945001,0.878561,0.622762,0.324981,0.677183
4,0.977484,0.206333,0.821440,0.304261,0.500000,0.632718,0.807579,0.456973,0.326972,0.653512
5,0.157804,0.827202,0.976000,0.054999,0.367282,0.500000,0.782276,0.193685,0.525629,0.428882
6,0.783962,0.162313,0.543134,0.121439,0.192421,0.217724,0.500000,0.871914,0.489886,0.411217
7,0.842357,0.088312,0.115450,0.377238,0.543027,0.806315,0.128086,0.500000,0.252482,0.765607
8,0.619542,0.469178,0.930551,0.675019,0.673028,0.474371,0.510114,0.747518,0.500000,0.538308
9,0.931187,0.931012,0.983889,0.322817,0.346488,0.571118,0.588783,0.234393,0.461692,0.500000


In [4]:
my_hand = np.random.randint(0, n, cards+1)
my_hand

array([0, 4, 4, 8, 3, 9])

In [5]:
enemy_cards = np.random.randint(0, n, cards)
enemy_cards

array([5, 6, 9, 1, 9])

In [6]:
def compire(first, second):
    out = 0
    for comb in itertools.product('10', repeat=cards):
        count_ones = 0
        for i in range(cards):
            count_ones += int(comb[i]=='1')
        if count_ones > cards // 2:
            mul = 1
            for i, item in enumerate(comb):
                if item == '1':
                    mul *= A[first[i], second[i]]
                else:
                    mul *= A[second[i], first[i]]
            out += mul
    return out
# compire(first, second)

In [7]:
argmax = list()
max_prob = 0
for my_cards in itertools.permutations(my_hand, 5):
    probability = compire(my_cards, enemy_cards)
    if max_prob < probability:
        max_prob = probability
        argmax = my_cards
argmax = list(argmax)
print('My hand:', my_hand)
print('My cards:', argmax)
print('Enemy cards:', enemy_cards)
print('Probability:', max_prob)
result = pd.DataFrame({'My card' : [i for i in argmax],
                       'Enemy card' : [i for i in enemy_cards],
                       'Result': [A[argmax[i], enemy_cards[i]] for i in range(cards)]})
result

My hand: [0 4 4 8 3 9]
My cards: [0, 3, 4, 9, 4]
Enemy cards: [5 6 9 1 9]
Probability: 0.9458781313331233


,My card,Enemy card,Result
0,0,5,0.842196
1,3,6,0.878561
2,4,9,0.653512
3,9,1,0.931012
4,4,9,0.653512


In [8]:
pd.DataFrame(A)

,0,1,2,3,4,5,6,7,8,9
0,0.500000,0.524621,0.251550,0.288951,0.022516,0.842196,0.216038,0.157643,0.380458,0.068813
1,0.475379,0.500000,0.157053,0.670516,0.793667,0.172798,0.837687,0.911688,0.530822,0.068988
2,0.748450,0.842947,0.500000,0.945454,0.178560,0.024000,0.456866,0.884550,0.069449,0.016111
3,0.711049,0.329484,0.054546,0.500000,0.695739,0.945001,0.878561,0.622762,0.324981,0.677183
4,0.977484,0.206333,0.821440,0.304261,0.500000,0.632718,0.807579,0.456973,0.326972,0.653512
5,0.157804,0.827202,0.976000,0.054999,0.367282,0.500000,0.782276,0.193685,0.525629,0.428882
6,0.783962,0.162313,0.543134,0.121439,0.192421,0.217724,0.500000,0.871914,0.489886,0.411217
7,0.842357,0.088312,0.115450,0.377238,0.543027,0.806315,0.128086,0.500000,0.252482,0.765607
8,0.619542,0.469178,0.930551,0.675019,0.673028,0.474371,0.510114,0.747518,0.500000,0.538308
9,0.931187,0.931012,0.983889,0.322817,0.346488,0.571118,0.588783,0.234393,0.461692,0.500000


In [9]:
%%time
argmax = list()
max_prob = 0
all_obs = 0
dict_hands = {}
for _ in itertools.combinations_with_replacement([i for i in range(n)], cards):
    all_obs += 1
for first in tqdm(itertools.combinations_with_replacement([i for i in range(n)], cards), total=all_obs):
    sum_prob = 0
    for second in itertools.combinations_with_replacement([i for i in range(n)], cards):
        sum_prob += compire(first, second)
    dict_hands[first] = sum_prob / all_obs
    if sum_prob > max_prob:
        max_prob = sum_prob
        argmax = first
print(argmax)
print(max_prob / all_obs)

100%|██████████| 2002/2002 [04:30<00:00,  7.41it/s]

(2, 8, 8, 8, 8)
0.7190748041278096
CPU times: user 4min 28s, sys: 1.81 s, total: 4min 29s
Wall time: 4min 30s


In [18]:
sorted_by_value = sorted(dict_hands.items(), key=lambda x: x[1], reverse=True)
sorted_by_value

[((2, 8, 8, 8, 8), 0.7190748041278096),
 ((2, 2, 3, 3, 3), 0.7164474352096429),
 ((4, 8, 8, 8, 8), 0.7037983232996813),
 ((2, 2, 3, 3, 8), 0.7036801769400416),
 ((8, 8, 8, 8, 8), 0.7030612862920113),
 ((2, 3, 3, 3, 3), 0.6979989728968232),
 ((2, 2, 3, 3, 4), 0.6954588041347538),
 ((2, 4, 8, 8, 8), 0.6917236309599593),
 ((2, 2, 3, 8, 8), 0.6903941850933577),
 ((2, 2, 8, 8, 8), 0.6885318391119298),
 ((2, 8, 8, 8, 9), 0.6861258549344176),
 ((2, 2, 3, 3, 7), 0.6856919102358354),
 ((2, 3, 3, 3, 8), 0.6840008626449893),
 ((2, 3, 8, 8, 8), 0.6823556157468897),
 ((2, 2, 3, 3, 6), 0.6769538959221493),
 ((2, 3, 3, 3, 4), 0.6762811040989664),
 ((2, 5, 8, 8, 8), 0.674759260003541),
 ((2, 2, 3, 4, 8), 0.6734617197718156),
 ((2, 3, 3, 8, 8), 0.6725038842406472),
 ((4, 4, 8, 8, 8), 0.6717137930570468),
 ((4, 8, 8, 8, 9), 0.6698759085921369),
 ((8, 8, 8, 8, 9), 0.6689473052626485),
 ((2, 2, 4, 8, 8), 0.6688282592570145),
 ((2, 2, 3, 3, 9), 0.6679389941822367),
 ((2, 3, 3, 3, 7), 0.6669737226091775),
 